In [298]:
import copy


In [879]:
class csp_variable:
    """
    dom debe ser un set, para que ninguna tupla que representa el dominio de la variable se repita.
    var_id es el identificador de la variable. Por defecto es None.
    """
    def __init__(self, var_id = None, dom = set()):
        self.id = var_id
        self.dom = set(dom)
        self.variable = (var_id, dom)
    
    def __repr__(self):
        return "{id}".format(id=self.id)
    
    '''
        Returns a copy of the domain.
    '''
    def Domain(self):
        return copy.deepcopy(self.dom)
    
    def getID(self):
        return self.id
    
    def modifyDomain(self, newDomain):
        self.dom = newDomain

In [947]:
class csp:
    def __init__(self):
        self.X = {} # k,v pairs where k is the variable identifier and v is the domain of k.
        self.D = self.getDomains() #k,v pair where k is X[i] and v is a set representing X[i]'s domain.
        self.C = [] #list of k,v tuples where k is the set of variables involved in the constraint, and v is the constraint.
                    # Every item in C is unique.
    
    def getArcs(self):
        self.updateDomains()
        return [Ci for Ci in self.C]
    
    def getAssignments(self):
        assigned = {Xi: Xi.Domain() for Xi in self.X if len(Xi.Domain()) == 1}
        return assigned
    
    def AssignValueTo(self, var, val):
        for Xi in self.X:
            if Xi == var:
                Xi.modifyDomain({val}) 
        self.updateDomains()
    
    '''
        Metodo que genera el grafo de restricciones del problema.
    '''
    def generateGraph(self):
        self.updateDomains()
        graph = dict()
        for Xi in self.X:
            # First we get the variable of which to get its neighbors
            #print('Selected Xi: {var}'.format(var = Xi))

            # We get the neighbors of Xi
            neighbors = list()

            # To get its neighbors we have to ask which variables are related to Xi in the constraints in C.
            #print('Constraints which involve Xi={0}:'.format(Xi))
            for constraint in self.C:
                if Xi in constraint:
                    constraint = set(constraint)
                    #print(constraint)
                    # For every constraint which involves Xi, we remove it and add it into a list of neighbors.
                    related_var = constraint - {Xi}
                    #print('The related variable to Xi={0} is {1}'.format(Xi, related_var))
                    # Once we get the related variable, we pop it off the related_var set to pass it as a regular object to a list
                    # To then have it hashed for uniqueness.
                    neighbors.append(related_var.pop())
                    graph[Xi] = neighbors
                
            #print('The neighbors of Xi={0} are: {1}'.format(Xi, neighbors))
        #print(graph)
        return graph
    
    '''
        Returns a copy of the list of neighbors of Xi
    '''
    def NeighborsOf(self, Xi):
        self.updateDomains()
        neighbors = self.generateGraph()[Xi]
        return neighbors
    
    def updateDomains(self):
        self.D = [{Xi: Xi.Domain()} for Xi in self.X]
    
    def defineVariables(self, X):
        self.X = X
        self.updateDomains()
        
    def defineConstraints(self, C):
        self.C = C
        self.updateDomains()
        
    def getVariables(self):
        return self.X
        
    def getDomains(self):
        self.updateDomains()
        return self.D
        
    def getConstraints(self):
        self.updateDomains()
        return self.C
    
    def __repr__(self):
        self.updateDomains()
        return "CSP: \n X ={vars},\n D ={domains},\n C ={constraints}\n".format(
            vars = self.X, 
            domains=self.D, 
            constraints=self.C)
    

In [953]:
"""
def Revise(csp, Xi, Xj):
    revised = False
    Di = Xi.Domain()
    Dj = Xj.Domain()
    
    # for each vk in Di
    # if no value vm in Dj allows (vk, vm) to satisfy the constraint between Xi and Xj then
    # remove vk from Di and set revised to true.
    
    # In order to accquire for vk the value vm which violates the constraint we remove the
    diff = [(vk ,Dj - {vk}) for vk in Di]
    print(diff)
    for tupl in diff:
        if len(tupl[1]) == 0:
            Di.remove(tupl[0])        
            
            revised = True
    return revised
"""

def Revise(csp, Xi, Xj):
    revised = False
    for vk in Xi.Domain():
        # Let's check in which order the key is in:
        #print('view of csp\'s constraints: {0}'.format(csp.C.keys()))
        if (Xi, Xj) in csp.C.keys():
            #print('({0},{1}) is a key of the constraint dict'.format(Xi, Xj))
            constraint = (Xi, Xj)
        elif (Xj, Xi) in csp.C.keys():
            #print('({0},{1}) is a key of the constraint dict'.format(Xj, Xi))
            constraint = (Xj, Xi)
        else:
            print('Something bad happened. ({0}, {1}) is not a key of csp.C'.format(Xi, Xj))
    
        if not any((vk, vm) in csp.C[constraint] for vm in Xj.Domain()):
            Xi.modifyDomain(Xi.Domain() - {vk})
            revised = True
    return revised
    

In [834]:
def AC3(csp):
    queue = csp.getArcs()
    graph = csp.generateGraph()
    
    while len(queue) != 0:
        (Xi, Xj) = queue.pop()
        #print('Constraint being evaluated: ({0},{1})'.format(Xi, Xj))
        if Revise(csp, Xi, Xj):
            #print('{0}\'s domain: {1}'.format(Xi, Xi.Domain()))
            #print('{0}\'s domain: {1}'.format(Xj, Xj.Domain()))
            if len(Xi.Domain()) == 0:
                return False
            #print('Xi={0}\'s neighbors: {1}'.format(Xi, csp.NeighborsOf(Xi)))
            #print('Xj={0}'.format(Xj))
            
            neighbors_without_Xj = set(csp.NeighborsOf(Xi)) - {Xj}
            if len(neighbors_without_Xj) == 0:
                break
            
            for Xk in neighbors_without_Xj:
                queue.append((Xk, Xi))

    return True

In [4]:
class value:
    def __init__(self):
        pass
    
    def IsConsistent(self, assignment):
        pass

In [866]:
"""
    Selects an arbitrary unassigned variable from the CSP and returns it;
    if no unassigned variable is found, returns None.
"""
def Select_Unassigned_Variable(csp):
    for Xi in csp.X:
        if len(Xi.Domain()) > 1:
            return Xi
    return None

In [1027]:
"""
    Ordering heuristic to be used.
    Least-Constraining-Value
    
    Returns the value that rules out the fewest choices for the neighboring variables in the constraint graph.
    Tailored specially for the map colouring problem. This heuristic is not general because I'm stupid :(
    
    TODO: make it general.
"""
def Order_Domain_Values(var, csp):
    sums = dict()
    print(csp.NeighborsOf(var))
    # For every value in var's domain
    for val in var.Domain():
        # This variable will aid in counting how many items in total the neighboring variables have
        # after asigning a value to var.
        count = 0
        for neighbor in csp.NeighborsOf(var):
            # We count how many possible values are left in a domain after asigning var a value.
            if val in neighbor.Domain():
                # TODO: Test if we could evaluate the constraint here and for every value
                # of the neighbors domain which violates a constraint given the assignment
                # made to var substract 1 from the neighbor's domain.
                
                # If val is in its neighbor domain, its only logical to remove said value 
                # according to the map colouring constraints.
                #print('conflicting value: {0}'.format(val))

                #print('neighbor: {0};\ndomain: {1}'.format(neighbor, neighbor.Domain()))
                count+= (len(neighbor.Domain())-1)
                #print(count)
            else:
                #if val is not in the neighbors domain, we still have to count the sum:
                count+=len(neighbor.Domain())
        sums[count]=val

    return [sums[key] for key in sorted(sums, reverse=True)]

In [1033]:
"""
    Order_Domain_Values testing grounds..
"""
test_problem = csp()

Di = set(['red', 'green', 'blue'])

WA = csp_variable('WA', Di)
NT = csp_variable('NT', Di)
Q = csp_variable('Q', Di)
NSW = csp_variable('NSW', Di)
V = csp_variable('V', Di)
SA = csp_variable('SA', Di)
T = csp_variable('T', Di)

test_problem.X = set([WA, NT, Q, NSW, V, SA, T])
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (WA, SA): {(vi, vj) for vi in WA.Domain() for vj in SA.Domain() if vi is not vj},
    (NT, SA): {(vi, vj) for vi in NT.Domain() for vj in SA.Domain() if vi is not vj},
    (SA, Q): {(vi, vj) for vi in SA.Domain() for vj in Q.Domain() if vi is not vj},
    (WA, NT): {(vi, vj) for vi in WA.Domain() for vj in NT.Domain() if vi is not vj},
    (NT, Q): {(vi, vj) for vi in NT.Domain() for vj in Q.Domain() if vi is not vj},
    (Q, NSW): {(vi, vj) for vi in Q.Domain() for vj in NSW.Domain() if vi is not vj}
}



#print(test_problem)
test_problem.AssignValueTo(WA, 'red')
test_problem.AssignValueTo(NT, 'green')
test_problem.AssignValueTo(SA, 'blue')
Revise(test_problem, Q, NT)

var = Q
"""
sums = dict()
print(test_problem.NeighborsOf(var))
for val in var.Domain():
    count = 0
    for neighbor in test_problem.NeighborsOf(var):
        if val in neighbor.Domain():
            print('conflicting value: {0}'.format(val))
            
            print('neighbor: {0};\ndomain: {1}'.format(neighbor, neighbor.Domain()))
            count+= (len(neighbor.Domain())-1)#/len(test_problem.NeighborsOf(var))
            print(count)
        else:
            #if val is not in the neighbors domain, we still have to count the sum:
            count+=len(neighbor.Domain())
    sums[count]=val

print(sums)

print(sums)
sort = [sums[key] for key in sorted(sums, reverse=True)]
print(sort)
"""

Order_Domain_Values(Q, test_problem)

[SA, NT, NSW]


['red', 'blue']

In [853]:
 """
    Assignment is a dictionary with variables and their assigned values. 
    Ex:
        CSP: 
         X ={b, a},
         D =[{b: {1, 2, 3}}, {a: {1, 2, 3}}],
         C ={(a, b): {(1, 2), (2, 1)}}
        
    An assignment would be:
        {a: 1, b:2} or {a:2} or {}
"""
def AssignmentComplete(assignment, csp):
    for Xi in csp.X:
        if Xi not in assignment.keys():
            return False
    return True

In [854]:
def Backtrack(assignment, csp):
    failure = none
    
    if AssignmentComplete(assignment, csp):
        return assignment
    
    var = Select_Unasigned_Variable(csp)
    
    for value in var.Domain():#Order_Domain_Values(var, assignment, csp):
        if value.IsConsistent(assignment):
            assignment.add(value)
            inferences = Inference(csp, var, value) #AC3(csp)?
            if inferences.IsNotFailure(): # if AC3(csp):
                assignment.add(inferences)
                result = Backtrack(assignment, csp)
                if result.IsNotFailure():
                    return result
        assignment.remove(value)
        assignment.remove(inferences)
    return failure

In [1005]:
"""
    Backtrack testing grounds
"""

test_problem = csp()

a = csp_variable('a', {1, 2, 3})
b = csp_variable('b', {1, 2, 3})

test_problem.X = {a, b}
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (a, b):{(vk, vm) for vk in a.Domain() for vm in b.Domain() if vk+vm == 3}
}

print(test_problem)

"""
Di = set(['red', 'green', 'blue'])

WA = csp_variable('WA', Di)
NT = csp_variable('NT', Di)
Q = csp_variable('Q', Di)
NSW = csp_variable('NSW', Di)
V = csp_variable('V', Di)
SA = csp_variable('SA', Di)
T = csp_variable('T', Di)

test_problem.X = set([WA, NT, Q, NSW, V, SA, T])
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (WA, SA): {(vi, vj) for vi in WA.Domain() for vj in SA.Domain() if vi is not vj},
    (NT, SA): {(vi, vj) for vi in NT.Domain() for vj in SA.Domain() if vi is not vj},
    (SA, Q): {(vi, vj) for vi in SA.Domain() for vj in Q.Domain() if vi is not vj},
    (NSW, SA): {(vi, vj) for vi in NSW.Domain() for vj in SA.Domain() if vi is not vj},
    (V, SA): {(vi, vj) for vi in V.Domain() for vj in SA.Domain() if vi is not vj},
    (WA, NT): {(vi, vj) for vi in WA.Domain() for vj in NT.Domain() if vi is not vj},
    (NT, Q): {(vi, vj) for vi in NT.Domain() for vj in Q.Domain() if vi is not vj},
    (Q, NSW): {(vi, vj) for vi in Q.Domain() for vj in NSW.Domain() if vi is not vj},
    (NSW, V): {(vi, vj) for vi in NSW.Domain() for vj in V.Domain() if vi is not vj}
}
"""

"""
a = csp_variable('a', {1, 2, 3})
b = csp_variable('b', {1, 2, 3})
c = csp_variable('c', {1, 2, 3})

test_problem.X = {a, b, c}
test_problem.D = [Xi.Domain() for Xi in test_problem.X]
test_problem.C = {
    (a, b):{(vk, vm) for vk in a.Domain() for vm in b.Domain() if vk+vm == 3},
    (b, c): {(vk, vm) for vk in b.Domain() for vm in c.Domain() if vk>vm},
    (a, c):{(vk, vm) for vk in a.Domain() for vm in c.Domain() if vk != vm}
}
"""

CSP: 
 X ={a, b},
 D =[{a: {1, 2, 3}}, {b: {1, 2, 3}}],
 C ={(a, b): {(1, 2), (2, 1)}}



"\na = csp_variable('a', {1, 2, 3})\nb = csp_variable('b', {1, 2, 3})\nc = csp_variable('c', {1, 2, 3})\n\ntest_problem.X = {a, b, c}\ntest_problem.D = [Xi.Domain() for Xi in test_problem.X]\ntest_problem.C = {\n    (a, b):{(vk, vm) for vk in a.Domain() for vm in b.Domain() if vk+vm == 3},\n    (b, c): {(vk, vm) for vk in b.Domain() for vm in c.Domain() if vk>vm},\n    (a, c):{(vk, vm) for vk in a.Domain() for vm in c.Domain() if vk != vm}\n}\n"

In [3]:
def Backtracking_Search(csp)
    return Backtrack([], csp)

set()
